### 🎯 What You'll Learn:
✅ Search and select Sentinel-2 satellite images using Earth Engine's Data Catalog

✅ Filter images by date, cloud coverage, and area of interest (AOI)

✅ Visualize true-color composites using RGB bands

✅ Export selected satellite images to Google Drive for further use



In [ ]:
# import libraries
import ee
import geemap

In [ ]:
# Authentication
ee.Authenticate()

# Initializing
ee.Initialize(project =  'ee-nouribrahim25'  )

In [ ]:
# Convert aoi (Delta Nile) coordinates into Polygon gemometry

delta_coors = [
  [29.476443, 29.830897],
  [29.476443, 31.615753],
  [32.805301, 31.615753],
  [32.805301, 29.830897],
  [29.476443, 29.830897]
  ]

aoi = ee.Geometry.Polygon(delta_coors)
aoi

In [ ]:
# Create the first interactive map
Map = geemap.Map( width = 800)

# Add aoi as a layer
Map.add_layer(aoi, {}, 'Delta Nile')
Map

###Load a Sentinel-2 image collection

In [ ]:
# We'll filter for a recent, cloud-free image over the Nile Delta.

s2_image = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
    .filterBounds(aoi)
    .filterDate('2024-01-01', '2025-01-01')
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
    .median()
)

s2_image

In [ ]:
# Select the 10-meter resolution bands (B2, B3, B4, B8) , B2: Blue / B3: Green / B4: Red / B8: Near-Infrared (NIR)
# We'll use B4 (Red) as our reference for getting the projection, as it's 10m.

s2_image = s2_image.select(['B2', 'B3', 'B4', 'B8'])

In [ ]:
# Get the projection (CRS and transform) for a 10-meter band.
# We use the B4 band's projection as it's natively 10m.

projection_10m = s2_image.select('B4').projection().getInfo()

export_crs = projection_10m['crs']
export_crs_transform = projection_10m['transform']


In [ ]:
print(f"Sentinel-2 CRS: {export_crs}")
print(f"Sentinel-2 CRS Transform (10m bands): {export_crs_transform}")

In [ ]:
# Visualize the true color aoi image
Map.addLayer(s2_image.select(['B4', 'B3', 'B2']), {'min': 0, 'max': 3000}, 'Sentinel-2 RGB')
Map

### Define and Start the Image Export Task

In [ ]:
# Define the export task
task = ee.batch.Export.image.toDrive(
    image= s2_image,
    description='nile_delta_s2_10m_precise',
    folder='GEE_Exports',
    fileNamePrefix='nile_delta_s2_10m_precise',
    crs = export_crs,
    crsTransform = export_crs_transform,
    region= aoi,
    maxPixels=1e10,                # Increase maxPixels for larger areas if needed
    fileFormat='GeoTIFF'           # GeoTIFF is recommended for geospatial data
)

# Start the export task
task.start()

### Exporting Each Band Separately

To export each band individually, you need to loop through the list of bands you want to export and create a separate export task for each one.

In [ ]:
# Assume s2_image is your selected image with bands ['B2', 'B3', 'B4', 'B8']
# and aoi, export_crs, export_crs_transform are defined from previous cells.

# List of bands to export separately
bands_to_export = ['B2', 'B3', 'B4', 'B8']

# Loop through each band and create an export task
for band_name in bands_to_export:
    print(f"Preparing to export band: {band_name}")

    # Select the current band
    single_band_image = s2_image.select(band_name)

    # Define the export task description and file name
    export_description = f'nile_delta_s2_band_{band_name}'
    file_name = f'nile_delta_s2_band_{band_name}'

    # Define the export task
    task = ee.batch.Export.image.toDrive(
        image=single_band_image,
        description=export_description,
        folder='GEE_Exports',              # Optional: specify a different folder
        fileNamePrefix=file_name,
        crs=export_crs,
        crsTransform=export_crs_transform,
        region=aoi,
        maxPixels=1e10,
        fileFormat='GeoTIFF'
    )

    # Start the export task
    task.start()

